# Hybrid GA-PSO algorithm on artificial test functions #

## Functions ##
The functions to optimize are the ones presented in the Artificial_test_functions notebook. For our mixed discrete-continuous case we consider minimization problems on functions of the form
$$ f: [a,b]^{d} \cap S^{d} \times [a,b]^{d} \to \mathbb{R}^+$$ 
    such that $a,b \in \mathbb{R}$ (usually the preferred search interval for each function) and $[a,b]^{d} \cap S^{d}$ is finite (e.g. $S=\mathbb{Z}$). More precisely we impose that $[a,b] \cap S$ is a finite set of equispaced points. \
 In the following images the case $D=4$ ($D$ is the total dimension, in our case $D=2d$) is depicted.:

<img src="./images/4D_ex.png" alt="alternative text" width="600"/>

The following image highlights how for every $\delta \in [a,b]^{d} \cap S^{d}$ we have a function $f_{\delta}:[a,b]^{d} \to \mathbb{R}^+$ that preserves some properties of the original not partially discretized function, for example continuity or differentiability.
<img src="./images/2D_ex.png" alt="alternative text" width="200"/>

Note that the case we consider here is **not** the most **general** possible: we are considering discrete sets that enjoy **total ordering** for each coordinate. The method presented below can also be adapted to unordered sets provided the mutation process is modified.

In [20]:
import math 
import numpy as np
import copy
import matplotlib.pyplot as plt
import random

In [21]:
#run the notebook where the test functions and some other utilities are defined
%run 01_Artificial_test_functions.ipynb

## The algorithm ##

The algorithm is an **hybrid** one that combines a **Genetic algorithm**-like part for the discrete part and a **Particle Swarm Optimization**-like part to handle the evolution of the continuous part.\
Through the whole algorithm an individual is an array of dimension $D=2d$ of the type $[discrete,continuous]=[discr_1,\dots,discr_d,cont_1,\dots,cont_d]$. The population is made up of $n$ individuals.\
The main processes involved in the algorithm are the following (more detailed information can be found in the code):
* **Initialization**: the population of $n\_pop$ individual is generated with each coordinate uniformly random in the search interval.
* **Evaluation**: for each individual the fitness is calculated.
* **Stakes**: we keep a list of best performing individuals that is as diverse as possibile. It has a constant number of element $stakes\_length$. \
Given a candidate individual $\gamma$ we consider the two closest individuals in the stakes, $\delta_1$ and $\delta_2$, according to the euclidean distance between their discrete parts. 
$$ dist_1=\min_{\delta \in stake}||\gamma_{discrete}-\delta_{discrete}||_2, \quad dist_2=\min_{\delta \in stake}||\gamma_{discrete}-\delta_{discrete}||_2 \text{ such that }||\gamma_{discrete}-\delta_{discrete}||_2>dist_1 $$
and $\delta_1$ and $\delta_2$ their respective argmin.\
If $f(\gamma)<f(\delta_1)$ and $dist_1<threshold$ the candidate $\gamma$ replace $\delta_1$ in the stakes. \
If this does not happen and $f(\gamma)<f(\delta_2)$, $dist_2<threshold$ and $dist_1>threshold/2$ we replace in the stake $\delta_2$ with $\gamma$.\
If neither of this cases happen but $\gamma$ has a better fitness than $3/4$ of the elements in the stakes and it is sufficiently distant to each stake (at least the $threshold$) it replaces the stake with the lowest fitness.\
These technical details are chosen to produce a list of "good" individuals that are spread in the discrete space as much as possibile. The reason for this list is that GA process like selection and crossover do not preserve the identity of the individual, making it impossible to correctly define the cognitive part of the velocity in the PSO part.
* **PSO**: for each individual in the population we consider its continuous part. The best individual overall is the continuous part of the individual with the highest fitness. The velocity cognitive component that is usually constructed using the best performing iteration of the individual in this case uses instead the continuous part of the individual in the stake with the closest euclidean distance. \
We execute $PSO\_its$ iterations of PSO before passing the population to the GA-inspired part of the algorithm.

<img src="./images/PSO.png" alt="alternative text" width="400"/>

* **Selection**: The top $n\_pop/5$ individuals with best fitness are automatically selected. We execute many three-individual tournaments among the all the population until we get a selected population of size $n\_pop/2$.

<img src="./images/selection.png" alt="alternative text" width="400"/>

* **Crossover**: let consider the selected population. The first $n\_pop/10$ individuals are preserved without modifications. From the whole selected population we extract two inviduals and we perform crossover among them. We iterate the process (with reinsertion) until we reach a population of size $n\_pop-n\_pop/10$.\
There are three types of crossover, selected uniformly at random:
 * *discrete-continuous crossover*: each resulting individual inherits the discrete part from a parent and the continuous part from the other.
 * *double one point crossover*: a one point crossover is performed for the discrete and the continuous part separately. The individuals are then combined. 
 * *uniform crossover*: two classic uniform crossover are performed, with a probability of $3/4$ and $1/4$ for the parents.
<img src="./images/crossovers.png" alt="alternative text" width="700"/>

* **Mutation**: The first $n\_pop/10$ elements are preserved without mutations. The remaining population is subjected to mutations of three types, uniformly at random:
 * *discrete-only $\epsilon$ mutation*: each discrete variable with probability $2/d$ is mutated by a small factor (sum or subtract $\epsilon$ or $2\epsilon$ where $\epsilon$ is the step between discrete values in our case)
 * *continuous-only $\epsilon$ mutation*: each continuous variable with probability $2/d$ is mutated by a small factor (sum or subtract a random value in the interval $[-|b-a|/5,|b-a|/5]$).
 * both of the above concurrently.
 
Then we add a mutated copy for each element of the first $n\_pop/10$ individuals to reach the total population of $n\_pop$.
 
  <img src="./images/mutations.png" alt="alternative text" width="550"/>

The unfolding of the processes described above can be summarised in the following diagram:

<img src="./images/diagram.png" alt="alternative text" width="600"/>

## Code ##

### Initialization functions ###

In [22]:
def discr_rand_gen(minimum, maximum, number, zero='true'):
    '''
    Generates uniformly at random discrete values
    
    in our case minimum=-maximum
    number is the number of slices we want to divide the interval in.
    zero is used to make sure that 0 is included. In our case this implies that we include the minimum of our opt problem 
    '''
    if zero=='true' and (number%2==0 or number<2):
        print('0 must be included')
        
    step=abs(minimum-maximum)/(number-1)    
    
    rnd=np.random.randint(0,number)
    
    res=minimum+rnd*step
    
    #returns a value in [minimum,maximum] with a step such that we have number possible values
    return res

In [23]:
def initialization(n,d_discr,d_cont,minimum,maximum,number):
    '''
    Initializes at random the population
    '''
    population=[]
    while len(population)<n:
        discrete=[]
        for i in range(0,d_discr):
            discrete+=[discr_rand_gen(minimum,maximum,number)]
           
        cont = [random.uniform(minimum, maximum) for _ in range(d_cont)]

        population += [discrete+cont]
            
    population=np.array(population)
    
    return population

In [24]:
def evaluation(func,pop,fit_eval,d_discr,d_cont,matrix,max_fitness_eval=100000):
    '''
    Evaluates the population fitness given a function
    
    matrix is the rotation matrix to apply
    it keeps count of the number of function evaluations
    '''
    
    values=[]
   
    for i in range(0,len(pop)):
        fit_eval+=1
        if fit_eval>=max_fitness_eval:
            break
        discrete = pop[i][:d_discr]
        cont = pop[i][d_discr:]
        values+=[func(discrete,cont,d_discr+d_cont,matrix)]

    #returns a list with the fitness values and the updated number of function evaluation    
    return values, fit_eval    

### Stakes ###

In [25]:
def list_of_stake_points(size,minimum,maximum,step,d_discr,d_cont):
    '''
    Generates a random set of stakes of a given size
    '''
    #generates random points in the search space
    stake=[]
    for i in range(size):
        element=[]
        for j in range(d_discr):
            element+=[discr_rand_gen(minimum,maximum, step)]
        cont = [random.uniform(minimum, maximum) for _ in range(d_cont)]

        element= [element+cont]
            
        stake+=element    
    
    #initialize their fitness at +inf
    st_val=[np.inf]*size
    
    #return the stakes and their values
    return stake,st_val        
    

In [26]:
def dist_stake(element, stake, d_discr, onlyfirst='no'):
    '''
    Calculates the distances between an elements and the stakes.
    Returns the two closest and their distances
    '''
    
    points_array=np.array(stake)
    #consider only the discrete part
    points_array_trunc = points_array[:, :d_discr]
    element_trunc = element[:d_discr]
    
    #calculate distances to all points
    distances = np.linalg.norm(points_array_trunc - element_trunc, axis=1)
    
    #find the indices of the two smallest distances
    closest_index = np.argmin(distances)
    d1=distances[closest_index]
    #set the closest distance to infinity to find the second closest
    distances[closest_index] = np.inf  
    second_closest_index = np.argmin(distances)

    #return the closest and second closest points
    closest_point = points_array[closest_index]
    second_closest_point = points_array[second_closest_index]
    
    if onlyfirst=='yes':
        return closest_index
    else:
        return closest_index, second_closest_index, d1, distances[second_closest_index]

In [27]:
def replace(element,el_value,stake,st_values,threshold,d_discr):
    '''
    Given an element and the list of stakes finds what stake must be replaced (if any)
    '''
    
    sub=0
    #find the two closest points and their distances
    first,second,d1,d2=dist_stake(element, stake, d_discr)
    
    #replace the closest?
    if st_values[first]>el_value and d1<threshold:
        st_values[first]=el_value
        stake[first]=element
        sub=1
        
    #replace the second closest?    
    if sub==0 and st_values[second]>el_value and d2<threshold and d1>threshold/2:
        st_values[second]=el_value
        stake[second]=element
        sub=1
    
    #ordering
    arr_val=np.array(st_values)
    arr_stake=np.array(stake)
    sorted_indices = np.argsort(np.array(arr_val))
    st_values = list(arr_val[sorted_indices])
    stake = list(arr_stake[sorted_indices])
    
    #is it not close but fit enough?
    if sub==0 and el_value<np.percentile(st_values, 25) and d1>threshold:
        #replace the worst
        stake[-1]=element
        st_values[-1]=el_value
    
    #return the updated stakes list and their values 
    return stake,st_values    

## PSO  ##

In [28]:
def PSO_stakes(n_it,popo,best_v,best_pos,values,function,rot_matrix,stake_PSO,stake_values,evals,d_discr,d_cont,
               max_fitness,threshold,w=0.6,c_soc=1,c_cog=1.4):
    '''
    PSO with the use of stakes
    '''
    #consider the continuous part of the best individual
    best_p=best_pos[-d_cont:]
    #augment the individuals with velocities for each continuous coordinate
    pop = np.hstack((popo, np.zeros((popo.shape[0], d_cont))))
    
    for j in range(0,n_it):
        for a in range(0,len(popo)):
            #closest individual in the stake (according to euclidean distance on the discrete part)
            cl_index=dist_stake(popo[a],stake_PSO,d_discr,onlyfirst='yes')
            cl_point=stake_PSO[cl_index]
            
            #random coefficients
            r_1_1=np.random.uniform(0,1)
            r_2_1=np.random.uniform(0,1)
            
            #update velocity vector
            for q in range(0,d_cont):
                index=q+d_discr+d_cont
                pos_index=q+d_discr
                pop[a][index]=w*pop[a][index]+c_soc*(r_1_1*(best_p[q]-pop[a][pos_index]))+c_cog*(r_2_1*(cl_point[pos_index]-pop[a][pos_index]))
            
            #update the position, evaluation is done only at the end of each PSO_iteration
            for q in range(d_discr,d_cont+d_discr):
                pop[a][q]=pop[a][q]+pop[a][q+d_cont]
        
        #no handling on outside the boundary individuals        
        
        #evaluations of the new individuals
        for k in range(0,len(popo)):
            evals+=1
            if evals> max_fitness:
                #print('Maximum number of evaluation reached')
                break
            v=function(pop[k][:d_discr],pop[k][d_discr:d_discr+d_cont],d_discr+d_cont,rot_matrix)
            values[k]=v
        
        pop_trunc = pop[:, :-d_cont]
        
        #update the stakes given the updated individuals
        for i in range(len(pop)):
            stake_PSO,stake_values=replace(pop_trunc[i],values[i],stake_PSO,stake_values,threshold,d_discr)
        
        #update the best value
        best_it_v=min(values)
        best_it_pos=pop_trunc[np.argmin(values)]
        if best_it_v<=best_v:
            best_p=best_it_pos[-d_cont:]
            best_v=best_it_v
    
    #returns the updated population, the list of values, the best value, the individual with the best fitness,
    #the stakes, their values and the number of function evaluation up to this point 
    return pop_trunc,values,min(values),pop_trunc[np.argmin(values)],stake_PSO,stake_values,evals

## Selection, crossover and mutation ## 

In [29]:
def selection(n_pop, population,values):
    '''
    Performs the described selection
    '''
    #sorting population-values based on values
    sort_indices = np.argsort(values)
    population_sorted = population[sort_indices]
    values_sorted = np.sort(values)
    
    #top 1/5 (approx)
    twenty = int(np.floor(n_pop/5))
    population_20 = population_sorted[:twenty]
    values_20 = values_sorted[:twenty]
    
    #tournament of size 3 among the rest of the elements. At the end of it we have half of the population
    ind=[]
    for _ in range(0,int(n_pop/2)-twenty):
            numbers = np.random.randint(twenty, n_pop, size=3)
            lower_number = np.min(numbers)
            ind+=[lower_number]
    
    #add the winners to the above 1/5
    population_keep=np.concatenate((population_20,population_sorted[ind]),axis=0)
    values_keep=np.concatenate((values_20,values_sorted[ind]),axis=0)
    
    #in the end two arrays of size n_pop/2, the first 1/5 is correctly ordered
    return population_keep,values_keep

In [30]:
def crossover(n_pop,population_k,d_discr,d_cont):
    '''
    Performs the described crossover
    '''
    #select the best performing 1/10
    #crossover population is already ordered, at least the first 1/5
    ten = int(np.floor(n_pop/10))
    cross_pop=population_k[:ten]
    
    #perform crossover
    while len(cross_pop)<n_pop-ten:
        #three type of crossover
        type_cross=np.random.randint(0, 3)
        #select parents uniformly among the selected population 
        
        
        mom = np.random.randint(0,len(population_k))
        dad = np.random.randint(0,len(population_k))
        
        if type_cross==0: #performs discrete-continuous crossover
            #discrete part from a parent, continuous part from the other
            son1 = np.concatenate((population_k[mom][:d_discr], population_k[dad][-d_cont:]))
            son2 = np.concatenate((population_k[dad][:d_discr], population_k[mom][-d_cont:]))
            
        if type_cross==1: #performs double one point crossover
            #extract two pints for crossover
            cross_point_discr = np.random.randint(0,d_discr)
            cross_point_cont = np.random.randint(0,d_cont)
            
            son1 = np.concatenate((population_k[mom][:cross_point_discr],population_k[dad][cross_point_discr:d_discr], 
                                   population_k[mom][d_discr:d_discr+cross_point_cont], population_k[dad][d_discr+cross_point_cont:]))
            son2 = np.concatenate((population_k[dad][:cross_point_discr],population_k[mom][cross_point_discr:d_discr],  
                                   population_k[dad][d_discr:d_discr+cross_point_cont], population_k[mom][d_discr+cross_point_cont:]))
        
        if type_cross==2: #performs uniform crossover
            for i in range(0,d_discr+d_cont):
                mom_copy = np.copy(population_k[mom])
                dad_copy = np.copy(population_k[dad])
                son1 = np.copy(population_k[mom])
                son2 = np.copy(population_k[dad])

                # perform uniform crossover with the probability 0.75
                for i in range(len(population_k[mom])):
                    if np.random.rand() < 0.75:
                        # Swap components if the random value is less than crossover_prob
                        son1[i]=dad_copy[i]
                    if np.random.rand() < 0.75:
                        # Swap components if the random value is less than crossover_prob
                        son2[i]=mom_copy[i]
                             
            
            
        cross_pop=np.concatenate((cross_pop,np.array([son1,son2])),axis=0)
           
    #cross pop should have exactly n_pop-n_pop/10 elements
    return cross_pop

In [31]:
def mutation(pop,n_pop,minimum,maximum,granularity,d_discr,d_cont):
    '''
    Performs the mutation
    '''
    not_mutated = int(np.floor(n_pop)/10)
    for index in range(0,len(pop)):
        #keep the first not_mutated elements without mutations
        if index>=not_mutated:
            
            #three types of mutations/perturbation
            type_mut=np.random.randint(0, 3)
            #only discrete part
            if type_mut==0:
                for var in range(0,d_discr):
                    is_changed = np.random.randint(0, 5)
                    if is_changed==1:
                        shift=np.random.randint(-2, 2)
                        pop[index][var]+=(abs(minimum-maximum)/(granularity-1))*shift 
                
            #only continuous part     
            if type_mut==1:
                for var in range(0,d_cont):
                    is_changed = type_mut=np.random.randint(0, 2)
                    if is_changed==1:
                        fo = np.random.randint(-(maximum-minimum)/5, (maximum-minimum)/5)
                        pop[index][d_discr+var]+=fo
                        
            #both discrete and continuous     
            if type_mut==2:
                for var in range(0,d_discr):
                    is_changed = np.random.randint(0, 5)
                    if is_changed==1:
                        shift=np.random.randint(-2, 2)
                        pop[index][var]+=(abs(minimum-maximum)/(granularity-1))*shift
                for var in range(0,d_cont):
                    is_changed = type_mut=np.random.randint(0, 2)
                    if is_changed==1:
                        fo = np.random.randint(-(maximum-minimum)/5, (maximum-minimum)/5)
                        pop[index][d_discr+var]+=fo
    
    for i in range(0,not_mutated):
        elite_mut = np.copy(pop[i])
        for var in range(0,d_discr):
            is_changed = np.random.randint(0, 5)
            if is_changed==1:
                shift=np.random.randint(-2, 2)
                elite_mut[var]=elite_mut[var]+(abs(minimum-maximum)/(granularity-1))*shift 
        pop=np.concatenate((pop, np.array([elite_mut])),axis=0)
    
    return pop

In [32]:
def GA_PSO_algorithm(epochs,n_it,function,rot_matrix,n_pop,d_discr,d_cont,minimum,maximum,granularity,stake_size,
                     dist_threshold,max_fitness_eval=100000,w=0.7,c_soc=0.9,c_cog=1.2):
    '''
    Main function. See algorithm description for more info.
    At the moment it outputs a list which contains the best value found up to a certain function evaluation,
    and a list that contains the number of function evaluations. Needed for the plots.
    '''
    #initialization
    population = initialization(n_pop,d_discr,d_cont,minimum,maximum,granularity)      
    
    #number of fitness evaluation
    f_eval = 0
    
    #evaluation
    values, f_eval = evaluation(function,population,f_eval,d_discr,d_cont,rot_matrix,max_fitness_eval)
    
    #store the best of the population    
    best_value, best = min(values),population[np.argmin(values)]
        
    #initialize the stakes
    stake,val_stake = list_of_stake_points(stake_size,minimum,maximum,granularity,d_discr,d_cont)
        
    #update the stakes given the initial population
    for i in range(len(population)):
        stake,val_stake = replace(population[i],values[i],stake,val_stake,dist_threshold,d_discr)
    
        
    tot_best=best_value
    
    #intialize the lists we need for the plot (num of function evaluation vs best value upt to that point)
    bmrk=[tot_best]
    f_evaluations=[f_eval]
    
    #main loop
    for epoch in range(0,epochs):
                      
        #PSO
        population,values,best_value,best,stake,val_stake,f_eval=PSO_stakes(n_it,population,best_value,best,values,function,
                                                                            rot_matrix,stake,val_stake,f_eval,d_discr,d_cont,
                                                                            max_fitness_eval,dist_threshold,
                                                                            w,c_soc,c_cog)
        
        if min(values)<tot_best:
                tot_best= min(values)
                
        #update the history of best value
        if epoch%4==0:
            f_evaluations+=[f_eval]
            bmrk+=[tot_best]
            
        #exit if too many function evaluations        
        if f_eval>=max_fitness_eval:
            return bmrk,f_evaluations

        #selection
        pop_keep,values_keep = selection(n_pop, population,values)
                 
        
        #crossover
        population = crossover(n_pop,pop_keep,d_discr,d_cont)
                
        
        #mutation
        population = mutation(population,n_pop,minimum,maximum,granularity,d_discr,d_cont)
        
        #evaluation
        values, f_eval = evaluation(function,population,f_eval,d_discr,d_cont,rot_matrix,max_fitness_eval)
        
        if min(values)<tot_best:
                tot_best= min(values)
                
        #update the history of best value
        if epoch%4==0:
            f_evaluations+=[f_eval]
            bmrk+=[tot_best]
            
        #exit if too many function evaluations        
        if f_eval>=max_fitness_eval:
            return bmrk,f_evaluations
    
    return bmrk,f_evaluations

In [19]:
#example of use
#GA_PSO_algorithm(500,8,alpine1_func_rotated,generate_random_rotation_matrix(50),50,25,25,-10,10,21,20,55,
#                 max_fitness_eval=100000)